In [1]:
import pyopencl as cl
import pyopencl.array as cl_array
import numpy as np
import numpy.linalg as la
import math
import torch
import torch.nn as nn
import torch.functional as F

In [2]:
%load_ext pyopencl.ipython_ext

In [3]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)
mf = cl.mem_flags

In [4]:
class LeNet(nn.Module):
    def __init__(self, num_classes):
        super(LeNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1 ),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fclayer = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,num_classes)
        ) 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x=x.view(-1, 16*5*5)
        x=self.fclayer(x)
        return x

model = LeNet(10)
model.load_state_dict(torch.load('model.pth'))
layer1_Conv2d = model.layer1[0]

In [5]:
layer1_Conv2d.weight.shape,layer1_Conv2d.bias.shape

(torch.Size([6, 1, 5, 5]), torch.Size([6]))

In [6]:
def conv2d(input_numpy, kernel_weight_numpy, kernel_bias_numpy, padding = 0):
    Ci, Hi, Wi = input_numpy.shape
    input_pad_numpy = torch.zeros(Ci, Hi+2*padding, Wi+2*padding)
    if padding > 0:
        input_pad_numpy[:, padding:-padding, padding:-padding] = input_numpy
    else:
        input_pad_numpy = input_numpy
    Ci, Hi, Wi = input_pad_numpy.shape
    Co, Ci, Hf, Wf = kernel_weight_numpy.shape
    Ho, Wo = Hi - Hf + 1, Wi - Wf + 1
    out = np.zeros((Co,Ho,Wo))
    # conv2d weight 7 loop
    for i in range(Ho):
        for j in range(Wo):
            for k in range(Co):
                for l in range(Hf):
                    for m in range(Wf):
                        for n in range(Ci):
                            out[k,i,j] += input_pad_numpy[n,i+l,j+m]*kernel_weight_numpy[k,n,l,m]
    for i in range(Ho):
        for j in range(Wo):
            for k in range(Co):
                out[k,i,j] += kernel_bias_numpy[k]
    return out

In [7]:
input_numpy = torch.randn(1,28,28)
weight_cpu = layer1_Conv2d.weight.detach().numpy()
bias_cpu = layer1_Conv2d.bias.detach().numpy()

padding = 2
Ci, Hi, Wi = input_numpy.shape
input_cpu = np.zeros((Ci, Hi+2*padding, Wi+2*padding)).astype(np.float32)
if padding > 0:
    input_cpu[:, padding:-padding, padding:-padding] = input_numpy
else:
    input_cpu = input_numpy
    
Ci, Hi, Wi = input_cpu.shape
Co, Ci, Hf, Wf = weight_cpu.shape
Ho, Wo = Hi - Hf + 1, Wi - Wf + 1
output_cpu = np.zeros((Co,Ho,Wo)).astype(np.float32)

In [8]:
input_gpu = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = input_cpu)

kernel_weight_gpu = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = weight_cpu)
kernel_bias_gpu = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = bias_cpu)

output_channel_gpu = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = np.int32(Co))
output_height_gpu = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = np.int32(Ho))
output_width_gpu = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = np.int32(Wo))
input_channel_gpu = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = np.int32(Ci))
input_height_gpu = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = np.int32(Hi))
input_width_gpu = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = np.int32(Wi))
feature_height_gpu = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = np.int32(Hf))
feature_width_gpu = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = np.int32(Wf))

output_gpu = cl.Buffer(ctx, mf.WRITE_ONLY, output_cpu.nbytes)

In [9]:
%%cl_kernel -o "-cl-fast-relaxed-math"

__kernel void Conv2D(__global const float *ift, 
                     __global float *weight, __global float *bias,
                     __global int *output_channel, __global int *output_height, __global int *output_width,
                     __global int *input_channel, __global int *input_height, __global int *input_width,
                     __global int *feature_height, __global int *feature_width,
                     __global float *oft)
{
    int Co = *output_channel, Ho = *output_height, Wo = *output_width;
    int Ci = *input_channel, Hi = *input_height, Wi = *input_width;
    int Hf = *feature_height, Wf = *feature_width;
    int posc = get_global_id(0), posh = get_global_id(1), posw = get_global_id(2);
    int So = Wo*Ho, Sf = Wf*Hf, Si = Wi*Hi;
    int Vf = Sf*Ci;
    int i = posc*(So) + (posh*Wo+posw);
    
    oft[i] = bias[posc];
    for(int l = 0; l < Hf; l++) {
        for(int m = 0; m < Wf; m++) {
            for(int n = 0; n < Ci; n++) {
                oft[i] += ift[(n*Si)+((posh+l)*Wi)+(posw+m)]*weight[(posc*Vf)+(n*Sf)+(l*Wf)+(m)];
            }
        }
    }
}

In [10]:
Conv2D(queue, input_cpu.shape, None, 
           input_gpu, 
           kernel_weight_gpu, kernel_bias_gpu,
           output_channel_gpu, output_height_gpu, output_width_gpu,
           input_channel_gpu, input_height_gpu, input_width_gpu,
           feature_height_gpu, feature_width_gpu,
           output_gpu)

In [11]:
cl.enqueue_copy(queue, output_cpu, output_gpu)

In [12]:
output_cpu[0,0,:]

array([-0.3761908 , -0.20601669,  0.57253695,  0.0436281 , -0.03303563,
        0.20020744,  0.6847333 ,  1.2288802 ,  0.39604503, -0.06561807,
       -0.7698254 ,  0.43003112,  0.43314782,  0.70534664,  0.437215  ,
       -0.2943088 , -0.85363436, -1.5228331 , -1.1102532 , -1.29766   ,
       -1.106854  ,  0.39128307,  1.0804993 ,  1.4540901 ,  1.6198981 ,
        1.0882974 ,  0.58785605,  0.20178679], dtype=float32)

In [13]:
np_res = conv2d(input_numpy, weight_cpu, bias_cpu, padding = 2)

In [14]:
np_res[0,0,:]

array([-0.37619083, -0.20601666,  0.57253699,  0.04362808, -0.03303564,
        0.2002074 ,  0.68473338,  1.22888015,  0.39604501, -0.06561805,
       -0.76982541,  0.43003111,  0.43314783,  0.70534657,  0.43721507,
       -0.29430877, -0.85363431, -1.522833  , -1.11025311, -1.29765989,
       -1.10685392,  0.39128314,  1.08049928,  1.4540901 ,  1.6198983 ,
        1.08829735,  0.58785592,  0.20178677])